# Classificador

In [12]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

Limpeza simples:

In [13]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

## Análise dos tweets marca Xiaomi

A análise foi baseada na classificação dos tweets, de forma binária, sendo favoráveis a marca (0), ou contra a marca (1)

In [14]:
import os

os.getcwd()

'C:\\Users\\User\\Desktop\\2ºsemestre\\Ciência dos dados\\P2_tweets'

In [15]:
tweets = pd.read_excel('xiaomi_1.xlsx')

In [16]:
tweets

,Treinamento,Classificação
0,@marcusc13 @xiaomibrasil @apple @motorolabr @s...,0
1,esse cel xiaomi é excelente! 😁,0
2,"@hugorambo18 não temos vagas, abandonou a famí...",1
3,@alysonhoffmann comprei 2 fones da xiaomi no d...,0
4,@itsramss aqui e team xiaomi ramenex junte-se ...,0
5,xiaomi eh a o anti cristo não caiam nessa http...,1
...,...,...
324,fui pesquisar sobre a xiaomi e gostei sla ache...,0
325,isso demonstra que muitos equipamentos são de ...,0
326,@mumudesp se quiser eu pego meus slides e te a...,0


In [17]:
tweets_idx = tweets.set_index('Treinamento')

In [18]:
tweets_idx

,Classificação
Treinamento,
"@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade, o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema. ela é pesada, lenta e com várias falhas. a xiaomi por exemplo, usa a miui, uma interface madura e bem otimizada.",0
esse cel xiaomi é excelente! 😁,0
"@hugorambo18 não temos vagas, abandonou a família xiaomi",1
"@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais, custavam quase 200 cada, no brasil, paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas.",0
@itsramss aqui e team xiaomi ramenex junte-se a nós,0
xiaomi eh a o anti cristo não caiam nessa https://t.co/ctkzhk3wlq,1
...,...
fui pesquisar sobre a xiaomi e gostei sla achei uma marca interessante,0
isso demonstra que muitos equipamentos são de baixa qualidade o meu celular está havaliado em 6 mil reais por causa da grande qualidade dos produtos shark e eu pago para a chinesa xiaomi fabricar aqui no brasil,0


Tweest a favor da marca:

In [37]:
afavor_tweets= tweets_idx.loc[(tweets_idx.Classificação==0)]

In [38]:
afavor_tweets

,Classificação
Treinamento,
"@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade, o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema. ela é pesada, lenta e com várias falhas. a xiaomi por exemplo, usa a miui, uma interface madura e bem otimizada.",0
esse cel xiaomi é excelente! 😁,0
"@hugorambo18 não temos vagas, abandonou a família xiaomi",0
"@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais, custavam quase 200 cada, no brasil, paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas.",0
@itsramss aqui e team xiaomi ramenex junte-se a nós,0
xiaomi eh a o anti cristo não caiam nessa https://t.co/ctkzhk3wlq,0
...,...
fui pesquisar sobre a xiaomi e gostei sla achei uma marca interessante,0
isso demonstra que muitos equipamentos são de baixa qualidade o meu celular está havaliado em 6 mil reais por causa da grande qualidade dos produtos shark e eu pago para a chinesa xiaomi fabricar aqui no brasil,0


In [39]:
afavor = pd.Series(afavor_tweets)

ValueError: Wrong number of items passed 1, placement implies 330

In [40]:
afavor

0    0
dtype: int64